<a href="https://colab.research.google.com/github/FrustratedBoy420/Project-Exhibition/blob/main/Diabetic_Patient_Model_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FrustratedBoy420/Project-Exhibition

Cloning into 'Project-Exhibition'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 37 (delta 15), reused 17 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 1.67 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0


In [4]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata # Corrected: Added this import
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json

from sentence_transformers import SentenceTransformer
from google.colab import userdata
import numpy as np
import faiss

In [14]:


# Step 1: GPU availability check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ----------------------------------
# 2. Device Setup
# ----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ----------------------------------
# 3. Load Model & Tokenizer (with quantization fix)
# ----------------------------------
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Changed to load_in_4bit=True
    bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
    bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    token=huggingface_token # Corrected use_auth_token to token
)

print("✅ Mistral-7B model and tokenizer loaded successfully.")

KeyboardInterrupt: 

In [ ]:
import json

file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"Data se {len(chunks)} chunks mil gaye.")

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:


# Embedding model ko load karo
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunks ka embeddings banao
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Embeddings successfully created.")
print(f"Pehle chunk ka embedding shape: {embeddings[0].shape}")

In [ ]:
import numpy as np
import faiss

In [ ]:


# Embeddings ko float32 format mein convert karo, jo FAISS ke liye zaruri hai
embeddings_np = np.array(embeddings).astype('float32')
dimension = embeddings_np.shape[1]

# FAISS index banao (yeh ek index hai jo fast search mein madad karta hai)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Index ko save karo taki dobara na banana pade
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")

print("FAISS index successfully created and saved.")

In [ ]:
# Yeh code tumhare existing code ke niche aayega
from google.colab import userdata

def get_recommendation_from_doc(user_query, chunks):
    # 1. User ke sawal ka embedding banao
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # 2. Vector database mein sabse relevant chunks dhoondo (top 3)
    k = 3
    distances, indices = index.search(query_embedding_np, k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # 3. Mistral-7B ke liye prompt banao
    prompt = f"""
    [INST]
    Based on the following medical guidelines from the American Diabetes Association, answer the user's question. Make sure your answer is helpful, accurate, and concise.

    Medical Guidelines: {context}

    User Question: {user_query}
    [/INST]
    """

    # 4. Model se jawab generate karvao
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# Ab isko test karo!
user_query = "What are the symptoms that a person have chances of having diabeties?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"Tumhara sawal: {user_query}")
print(f"Model ka jawab: {recommendation}")

##RAG TUNING


In [ ]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

In [ ]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------

import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json


In [ ]:
# # ----------------------------------
# # 2. Device Setup
# # ----------------------------------
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"✅ Using device: {device}")

# # ----------------------------------
# # 3. Load Model & Tokenizer (with quantization fix)
# # ----------------------------------
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# # Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Changed to load_in_4bit=True
#     bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
#     bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
#     llm_int8_threshold=6.0,
#     llm_int8_has_fp16_weight=True
# )

# tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=bnb_config,
#     token=huggingface_token # Corrected use_auth_token to token
# )

# print("✅ Mistral-7B model and tokenizer loaded successfully.")

In [ ]:
# ----------------------------------
# 4. Load Knowledge Base (JSON file)
# ----------------------------------
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"✅ Loaded {len(chunks)} chunks from knowledge base")

# ----------------------------------
# 5. Create / Load FAISS Index
# ----------------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings_np = np.array(embeddings).astype('float32')

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save index
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")
print("✅ FAISS index successfully created and saved.")

In [ ]:
# ----------------------------------
# 6. RAG Function
# ----------------------------------
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model - access device from a parameter
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Convert tensors to FP16 if the model is in FP16, EXCEPT for input_ids
    if model.dtype == torch.float16:
         inputs = {k: (v.half() if k != 'input_ids' else v) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# ----------------------------------
# 7. Test Query
# ----------------------------------
# Assuming `loaded_chunks`, `loaded_embedding_model`, `loaded_index`,
# `loaded_model`, and `loaded_tokenizer` are available from previous cells.
# Make sure to run the cells that load these components before running this cell.

user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

In [ ]:
# Save the model and tokenizer
# model_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
# tokenizer_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(tokenizer_save_path)

# print(f"Model saved to: {model_save_path}")
# print(f"Tokenizer saved to: {tokenizer_save_path}")

# Example of how to load in another file:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# #
model_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_tokenizer"

loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path)
loaded_model = AutoModelForCausalLM.from_pretrained(model_load_path)

print("Model and tokenizer loaded successfully in the new file.")

In [ ]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import json


print("✅ Model and tokenizer loaded successfully.")

# 1. Load the FAISS index
index_load_path = "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin"
loaded_index = faiss.read_index(index_load_path)
print("✅ FAISS index loaded successfully.")

# 2. Load the chunks
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
loaded_chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            loaded_chunks.append(item['text'])
print(f"✅ Loaded {len(loaded_chunks)} chunks from knowledge base")

# 3. Load the embedding model
from sentence_transformers import SentenceTransformer
loaded_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded successfully.")


# 4. Define the RAG Function using loaded components
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()

    return response_text

# 5. Test Query with loaded components
user_query = "What are the precautions for diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

In [ ]:
# 5. Test Query with loaded components
user_query = "What are the precautions for prediabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

# =========================================
# PART 2 — QLoRA Fine-tuning + RAG Inference (OOM Safe)
# =========================================

In [1]:
!pip install trl

In [2]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes

In [6]:
# Run this cell first in your fresh runtime
!pip install -U transformers datasets accelerate peft trl bitsandbytes

In [8]:
# FINAL CODE - RUN AFTER FACTORY RESET AND LIBRARY INSTALLATION
# -----------------------------------------------------------------------------

import torch
import os
import json
import math
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from google.colab import userdata

# Define paths and parameters
JSON_PATH = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
OUTPUT_DIR = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"

# Paths to your pre-saved model and tokenizer
model_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# Training parameters
EPOCHS = 1
LR = 0.0002
BATCH_SIZE = 1
GRAD_ACCUM = 32
MAX_SEQ_LEN = 256

# -----------------------------
# 1) Device Setup
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# -----------------------------
# 2) Load YOUR Saved Model & Tokenizer (with quantization)
# -----------------------------
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')

# Quantization config for loading the model efficiently
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading saved model from: {model_load_path}")
model = AutoModelForCausalLM.from_pretrained(
    model_load_path,
    quantization_config=bnb_config,
    dtype=torch.float16,
    device_map="auto",
    token=huggingface_token
)

print(f"Loading saved tokenizer from: {tokenizer_load_path}")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path, token=huggingface_token)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

print("✅ Saved model and tokenizer loaded.")

# -----------------------------
# 3) Load training data
# -----------------------------
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

records = []
for item in raw:
    text = item.get("text", "").strip()
    if not text:
        continue
    instruction = "Learn these diabetes guidelines to answer patient questions accurately."
    records.append({
        "instruction": instruction,
        "input": text,
        "output": "Acknowledged. I will use this guidance to answer future questions."
    })
print(f"Loaded {len(records)} training examples")
dataset = Dataset.from_list(records).train_test_split(test_size=0.05, seed=42)

# -----------------------------
# 4) Prompt formatting
# -----------------------------
def format_example(ex):
    user = f"{ex['instruction']}\n\nGuidelines:\n{ex['input']}\n\nTask: Confirm understanding."
    assistant = ex["output"]
    return f"[INST] {user} [/INST] {assistant}"

def map_to_text(batch):
    texts = [format_example({key: batch[key][i] for key in batch}) for i in range(len(batch['instruction']))]
    return {"text": texts}

train_texts = dataset["train"].map(map_to_text, batched=True, remove_columns=dataset["train"].column_names)

# -----------------------------
# 5) Training arguments
# -----------------------------
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    logging_steps=10,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    fp16=True,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
)

# -----------------------------
# 6) Initialize Trainer
# -----------------------------
# This is the correct signature for modern TRL library versions
trainer = SFTTrainer(
    model=model,
    args=args,
    peft_config=lora_config,
    train_dataset=train_texts,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=MAX_SEQ_LEN,
)

# -----------------------------
# 7) Train + Save
# -----------------------------
print("Starting continued training...")
trainer.train()

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ NEW LoRA adapters from this training run saved to: {OUTPUT_DIR}")

✅ Using device: cuda
Loading saved model from: /content/drive/MyDrive/LLM Model/mistral_rag_model


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading saved tokenizer from: /content/drive/MyDrive/LLM Model/mistral_rag_tokenizer
✅ Saved model and tokenizer loaded.
Loaded 638 training examples


Map:   0%|          | 0/606 [00:00<?, ? examples/s]

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'

In [5]:
# Cell 1
!pip install -U transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 14.9 MB/s eta 0:00:00


In [6]:
# Cell 2
import transformers
import peft
import trl
import torch
import accelerate
import bitsandbytes
import inspect

print("--- Library Versions ---")
print(f"Transformers: {transformers.__version__}")
print(f"PEFT:         {peft.__version__}")
print(f"TRL:          {trl.__version__}")
print(f"Accelerate:   {accelerate.__version__}")
print(f"BitsAndBytes: {bitsandbytes.__version__}")
print(f"PyTorch:      {torch.__version__}")

print("\n--- SFTTrainer Expected Arguments ---")
# This inspects the SFTTrainer to show us its required arguments
try:
    print(inspect.getfullargspec(trl.SFTTrainer.__init__))
except Exception as e:
    print(f"Could not inspect SFTTrainer: {e}")

--- Library Versions ---
Transformers: 4.56.1
PEFT:         0.17.1
TRL:          0.22.2
Accelerate:   1.10.1
BitsAndBytes: 0.47.0
PyTorch:      2.8.0+cu126

--- SFTTrainer Expected Arguments ---
FullArgSpec(args=['self', 'model', 'args', 'data_collator', 'train_dataset', 'eval_dataset', 'processing_class', 'compute_loss_func', 'compute_metrics', 'callbacks', 'optimizers', 'optimizer_cls_and_kwargs', 'preprocess_logits_for_metrics', 'peft_config', 'formatting_func'], varargs=None, varkw=None, defaults=(None, None, None, None, None, None, None, None, (None, None), None, None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={'model': typing.Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], 'args': typing.Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType], 'data_collator': typing.Optional[transformers.data.data_collator.DataCollator], 'train_dataset': typing.Union[datasets.arrow_dataset.Dataset

In [ ]:
# FINAL CODE - WITH OOM AND DEVICE ERROR FIXES
# -----------------------------------------------------------------------------

import torch
import gc
import os
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from google.colab import userdata

# Define paths and parameters
JSON_PATH = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
OUTPUT_DIR = "/content/drive/MyDrive/LLM Model/mistral_finetuned_rag_adapters_v2"

# Paths to your pre-saved model and tokenizer
model_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# Training parameters
EPOCHS = 1
LR = 0.0002
BATCH_SIZE = 1
GRAD_ACCUM = 32
MAX_SEQ_LEN = 192

# -----------------------------
# 1) Device Setup & Model Loading
# -----------------------------
# This check is important to determine the device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

huggingface_token = userdata.get('HUGGING_FACE_TOKEN')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading saved model from: {model_load_path}")
model = AutoModelForCausalLM.from_pretrained(
    model_load_path,
    quantization_config=bnb_config,
    dtype=torch.float16,
    # --- DEVICE FIX: Removed explicit device_map ---
    # Let accelerate handle device placement during training
    token=huggingface_token
)

print(f"Loading saved tokenizer from: {tokenizer_load_path}")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path, token=huggingface_token)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

tokenizer.model_max_length = MAX_SEQ_LEN
tokenizer.padding_side = 'right'

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

print("✅ Saved model and tokenizer loaded and configured.")

# -----------------------------
# 2) Load & Format Data
# -----------------------------
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

records = []
for item in raw:
    text = item.get("text", "").strip()
    if not text:
        continue
    instruction = "Learn these diabetes guidelines to answer patient questions accurately."
    records.append({
        "instruction": instruction,
        "input": text,
        "output": "Acknowledged. I will use this guidance to answer future questions."
    })
print(f"Loaded {len(records)} training examples")
dataset = Dataset.from_list(records)

def format_example(example):
    user = f"{example['instruction']}\n\nGuidelines:\n{example['input']}\n\nTask: Confirm understanding."
    assistant = example["output"]
    return f"[INST] {user} [/INST] {assistant}"

# -----------------------------
# 3) Training Arguments
# -----------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    gradient_checkpointing=True,
    learning_rate=LR,
    logging_steps=10,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    fp16=True,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
)

# -----------------------------
# 4) Initialize Trainer
# -----------------------------
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
    processing_class=tokenizer,
    formatting_func=format_example,
)

# --- Memory Cleanup ---
del raw, records, dataset
gc.collect()
torch.cuda.empty_cache()
print("Cleaned up memory before training.")

# -----------------------------
# 5) Train
# -----------------------------
print("Starting continued training with fixes...")
trainer.train()

# -----------------------------
# 6) Save
# -----------------------------
trainer.save_model(OUTPUT_DIR)

✅ Using device: cuda
Loading saved model from: /content/drive/MyDrive/LLM Model/mistral_rag_model


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading saved tokenizer from: /content/drive/MyDrive/LLM Model/mistral_rag_tokenizer
✅ Saved model and tokenizer loaded and configured.
Loaded 638 training examples


Applying formatting function to train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 192). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/638 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Cleaned up memory before training.
Starting continued training with fixes...


Step,Training Loss
